![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Merging Annotations From Multiple Named Entity Recognition Models

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.5.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.8:7077 
 AppName 
 Databricks Shell

In [0]:
# Sample data
data_chunk_merge = spark.createDataFrame([
  (1,"""A 63 years old man presents to the hospital with a history of recurrent infections that include cellulitis, pneumonias, and upper respiratory tract infections. He reports subjective fevers at home along with unintentional weight loss and occasional night sweats. The patient has a remote history of arthritis, which was diagnosed approximately 20 years ago and treated intermittently with methotrexate (MTX) and prednisone. On physical exam, he is found to be febrile at 102°F, rather cachectic, pale, and have hepatosplenomegaly. Several swollen joints that are tender to palpation and have decreased range of motion are also present. His laboratory values show pancytopenia with the most severe deficiency in neutrophils.
""")]).toDF("id","text")

data_chunk_merge.show(truncate=150)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------+
 id| text|
+---+------------------------------------------------------------------------------------------------------------------------------------------------------+
 1|A 63 years old man presents to the hospital with a history of recurrent infections that include cellulitis, pneumonias, and upper respiratory tract...|
+---+------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_deid_large", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "clinical_ner"]) \
    .setOutputCol("clinical_ner_chunk")

# internal clinical NER (general terms)
jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("jsl_ner")

jsl_ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "jsl_ner"]) \
    .setOutputCol("jsl_ner_chunk")

# merge ner_chunks by prioritizing the overlapping indices (chunks with longer lengths and highest information will be kept from each ner model)
chunk_merger = ChunkMergeApproach()\
    .setInputCols('clinical_ner_chunk', "jsl_ner_chunk")\
    .setOutputCol('merged_ner_chunk')

# merge ner_chunks regardess of overlapping indices 
# only works with 2.7 and later 
chunk_merger_NonOverlapped = ChunkMergeApproach()\
    .setInputCols('clinical_ner_chunk', "jsl_ner_chunk")\
    .setOutputCol('nonOverlapped_ner_chunk')\
    .setMergeOverlapping(False)


nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    clinical_ner_converter,
    jsl_ner,
    jsl_ner_converter,
    chunk_merger,
    chunk_merger_NonOverlapped])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_large download started this may take some time.
Approximate size to download 14.1 MB
[ | ][ / ][ — ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][ — ][OK!]

In [0]:
merged_data = model.transform(data_chunk_merge).cache()

In [0]:
from pyspark.sql import functions as F

result_df = merged_data.select('id',F.explode('merged_ner_chunk').alias("cols")) \
                       .select('id',F.expr("cols.begin").alias("begin"),
                               F.expr("cols.end").alias("end"),
                               F.expr("cols.result").alias("chunk"),
                               F.expr("cols.metadata.entity").alias("entity"))

result_df.show(50, truncate=100)

+---+-----+---+----------------------------------+-------------------------+
 id|begin|end| chunk| entity|
+---+-----+---+----------------------------------+-------------------------+
 1| 2| 13| 63 years old| Age|
 1| 15| 17| man| Gender|
 1| 35| 42| hospital| Clinical_Dept|
 1| 62| 70| recurrent| Modifier|
 1| 72| 81| infections|Disease_Syndrome_Disorder|
 1| 96|105| cellulitis|Disease_Syndrome_Disorder|
 1| 108|117| pneumonias|Disease_Syndrome_Disorder|
 1| 124|157|upper respiratory tract infections|Disease_Syndrome_Disorder|
 1| 160|161| He| Gender|
 1| 171|180| subjective| Modifier|
 1| 182|187| fevers| VS_Finding|
 1| 208|232| unintentional weight loss| Symptom|
 1| 238|247| occasional| Modifier|
 1| 249|260| night sweats| Symptom|
 1| 299|307| arthritis|Disease_Syndrome_Disorder|
 1| 344|355| 20 years ago| RelativeDate|
 1| 389|400| methotrexate| Drug_Ingredient|
 1| 403|405| MTX| Drug_Ingredient|
 1| 412|421| prednisone| Drug_Ingredient|
 1| 442|443| he| Gender|
 1| 460|466| febrile| VS_Finding|
 1| 485|493| cachectic| Symptom|
 1| 496|499| pale| Symptom|
 1| 511|528| hepatosplenomegaly| Symptom|
 1| 531|537| Several| Modifier|
 1| 539|552| swollen joints| Symptom|
 1| 563|568| tender| Symptom|
 1| 592|616| decreased range of motion| Symptom|
 1| 636|638| His| Gender|
 1| 663|674| pancytopenia|Disease_Syndrome_Disorder|
 1| 690|695| severe| Modifier|
 1| 697|721| deficiency in neutrophils|Disease_Syndrome_Disorder|
+---+-----+---+----------------------------------+-------------------------+

## NonOverlapped Chunk

All the entities form each ner model will be returned one by one

In [0]:
from pyspark.sql import functions as F

result_df2 = merged_data.select('id',F.explode('nonOverlapped_ner_chunk').alias("cols")) \
.select('id',F.expr("cols.begin").alias("begin"),
        F.expr("cols.end").alias("end"),
        F.expr("cols.result").alias("chunk"),
        F.expr("cols.metadata.entity").alias("entity"))

result_df2.show(50, truncate=100)


+---+-----+---+----------------------------------+-------------------------+
 id|begin|end| chunk| entity|
+---+-----+---+----------------------------------+-------------------------+
 1| 2| 13| 63 years old| Age|
 1| 2| 3| 63| AGE|
 1| 15| 17| man| Gender|
 1| 35| 42| hospital| Clinical_Dept|
 1| 62| 70| recurrent| Modifier|
 1| 72| 81| infections|Disease_Syndrome_Disorder|
 1| 96|105| cellulitis|Disease_Syndrome_Disorder|
 1| 108|117| pneumonias|Disease_Syndrome_Disorder|
 1| 124|157|upper respiratory tract infections|Disease_Syndrome_Disorder|
 1| 160|161| He| Gender|
 1| 171|180| subjective| Modifier|
 1| 182|187| fevers| VS_Finding|
 1| 208|232| unintentional weight loss| Symptom|
 1| 238|247| occasional| Modifier|
 1| 249|260| night sweats| Symptom|
 1| 299|307| arthritis|Disease_Syndrome_Disorder|
 1| 344|355| 20 years ago| RelativeDate|
 1| 389|400| methotrexate| Drug_Ingredient|
 1| 403|405| MTX| Drug_Ingredient|
 1| 412|421| prednisone| Drug_Ingredient|
 1| 442|443| he| Gender|
 1| 460|466| febrile| VS_Finding|
 1| 485|493| cachectic| Symptom|
 1| 496|499| pale| Symptom|
 1| 511|528| hepatosplenomegaly| Symptom|
 1| 531|537| Several| Modifier|
 1| 539|552| swollen joints| Symptom|
 1| 563|568| tender| Symptom|
 1| 592|616| decreased range of motion| Symptom|
 1| 636|638| His| Gender|
 1| 663|674| pancytopenia|Disease_Syndrome_Disorder|
 1| 690|695| severe| Modifier|
 1| 697|721| deficiency in neutrophils|Disease_Syndrome_Disorder|
+---+-----+---+----------------------------------+-------------------------+

## ChunkMergeApproach to admit N input cols

We can feed the ChunkMergerApproach more than 2 chunks. Also, we can filter out the entities that we don't want to get from the ChunkMergeApproach using `setBlackList ` parameter.

In [0]:
sample_text = """A 28 year old female with a history of gestational diabetes mellitus diagnosed eight years prior to 
presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis 
three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index 
( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting.
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . 
She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was 
significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , 
or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , 
anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin 
( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed 
as blood samples kept hemolyzing due to significant lipemia .
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior 
to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , 
the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , 
and lipase was 52 U/L .
 β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged 
 and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . 
 The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides 
 to 1400 mg/dL , within 24 hours .
 Twenty days ago.
 Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . 
 At birth the typical boy is growing slightly faster than the typical girl, but the velocities become equal at about 
 seven months, and then the girl grows faster until four years. 
 From then until adolescence no differences in velocity 
 can be detected. 21-02-2020 
21/04/2020
"""


In [0]:
%fs mkdirs file:/dbfs/data

res0: Boolean = true

In [0]:
# Defining ContextualParser for feeding ChunkMergerApproach

#defining rules
date = {
  "entity": "Parser_Date",
  "ruleScope": "sentence",
  "regex": "\\d{1,2}[\\/\\-\\:]{1}(\\d{1,2}[\\/\\-\\:]{1}){0,1}\\d{2,4}",
  "valuesDefinition":[],
  "prefix": [],
  "suffix": [],
  "contextLength": 150,
  "context": []
}


with open('/dbfs/data/date.json', 'w') as f:
    json.dump(date, f)


age = {
  "entity": "Parser_Age",
  "ruleScope": "sentence",
  "matchScope":"token",
  "regex" : "^[1][0-9][0-9]|[1-9][0-9]|[1-9]$",
  "prefix":["age of", "age"],
  "suffix": ["-years-old",
             "years-old",
             "-year-old",
             "-months-old",
             "-month-old",
             "-months-old",
             "-day-old",
             "-days-old",
             "month old",
             "days old",
             "year old",
             "years old", 
             "years",
             "year", 
             "months", 
             "old"
              ],
  "contextLength": 25,
  "context": [],
  "contextException": ["ago"],
  "exceptionDistance": 10
}

with open("/dbfs/data/age.json", 'w') as f:
  json.dump(age, f)



In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Contextual parser for age 
age_contextual_parser = ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("entity_age") \
    .setJsonPath("/dbfs/data/age.json") \
    .setCaseSensitive(False) \
    .setPrefixAndSuffixMatch(False)

chunks_age= ChunkConverter()\
    .setInputCols("entity_age")\
    .setOutputCol("chunk_age")

# Contextual parser for date
date_contextual_parser = ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("entity_date") \
    .setJsonPath("/dbfs/data/date.json") \
    .setCaseSensitive(False) \
    .setPrefixAndSuffixMatch(False)

chunks_date = ChunkConverter().setInputCols("entity_date").setOutputCol("chunk_date")

# Clinical word embeddings 
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# Extracting entities by ner_deid_large
ner_model = MedicalNerModel.pretrained("ner_deid_large","en","clinical/models") \
    .setInputCols("sentence","token","embeddings") \
    .setOutputCol("ner")

ner_converter= NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setWhiteList(["DATE", "AGE"])

# Chunkmerger; prioritize age_contextual_parser
parser_based_merge= ChunkMergeApproach()\
    .setInputCols(["chunk_age", "chunk_date", "ner_chunk"])\
    .setOutputCol("merged_chunks")

# Chunkmerger; prioritize ner_chunk
ner_based_merge= ChunkMergeApproach()\
    .setInputCols(["ner_chunk", "chunk_age", "chunk_date"])\
    .setOutputCol("merged_chunks_2")

# Using black list for limiting the entity types that will be extracted
limited_merge= ChunkMergeApproach()\
    .setInputCols(["ner_chunk", "chunk_age", "chunk_date"])\
    .setOutputCol("merged_chunks_black_list")\
    .setBlackList(["DATE", "Parser_Date"]) # this will block the dates. 

pipeline= Pipeline(stages=[
                           documentAssembler,
                           sentenceDetector,
                           tokenizer,
                           age_contextual_parser,
                           chunks_age,
                           date_contextual_parser,
                           chunks_date,
                           word_embeddings,
                           ner_model,
                           ner_converter,
                           parser_based_merge,
                           ner_based_merge,
                           limited_merge
])

empty_df= spark.createDataFrame([[""]]).toDF("text")
model= pipeline.fit(empty_df)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_large download started this may take some time.
Approximate size to download 14.1 MB
[ | ][OK!]

In [0]:
lmodel= LightPipeline(model)
lresult= lmodel.fullAnnotate(sample_text)[0]

In [0]:
lresult.keys()

Out[11]: dict_keys(['chunk_age', 'document', 'ner_chunk', 'token', 'entity_date', 'ner', 'merged_chunks_2', 'entity_age', 'merged_chunks_black_list', 'embeddings', 'chunk_date', 'sentence', 'merged_chunks'])

In [0]:
chunk= []
parser_based_merge= []
ner_based_merge= []

for i, k in list(zip(lresult["merged_chunks"], list(lresult["merged_chunks_2"],))):
  parser_based_merge.append(i.metadata["entity"])
  ner_based_merge.append(k.metadata["entity"])
  chunk.append(i.result)

df= pd.DataFrame({"chunk": chunk,"parser_based_merged_entity": parser_based_merge, "ner_based_merged_entity": ner_based_merge})
df.head()

Out[12]:

,chunk,parser_based_merged_entity,ner_based_merged_entity
0,28,Parser_Age,AGE
1,21-02-2020,Parser_Date,DATE
2,21/04/2020,Parser_Date,DATE


Checking the **BlackList** applied merger results. We've excluded "DATE" entities.

In [0]:
chunk= []
limited_merge_entity= []

for i in list(lresult["merged_chunks_black_list"]):
  chunk.append(i.result)
  limited_merge_entity.append(i.metadata["entity"])

df= pd.DataFrame({"chunk": chunk, "limited_entity": limited_merge_entity }) 
df.head()

Out[13]:

,chunk,limited_entity
0,28,AGE
